In [26]:
from typing import Dict, List
import pandas as pd
import datetime as dt
import os
import re
import json
from dotenv import load_dotenv
import pandas as pd

from specklepy.api.client import SpeckleClient
from specklepy.api.credentials import get_default_account
from specklepy.objects import Base
from specklepy.objects.geometry import Point, Line
from specklepy.transports.server import ServerTransport
from specklepy.api import operations

In [27]:
speckle_server = "app.speckle.systems"
speckle_token = os.environ["TOKEN"]
client = SpeckleClient(host=speckle_server)
client.authenticate_with_token(speckle_token)
stream_id = "013aaf06e7"
transport = ServerTransport(client=client, stream_id=stream_id)

In [28]:
top_granite_id = "d7a1881121fc1586211c4f2b2e0b6022"

obj = operations.receive(top_granite_id, transport)


In [29]:
obj

Base(id: d7a1881121fc1586211c4f2b2e0b6022, speckle_type: Base, totalChildrenCount: 63875)

In [30]:
from specklepy.serialization.base_object_serializer import BaseObjectSerializer


def to_dict(obj_id, transport):
    obj = operations.receive(obj_id, transport)
    serializer = BaseObjectSerializer()
    _, data = serializer.traverse_base(obj)
    return data

In [31]:
top_weathered_granite_data = to_dict(top_granite_id, transport)

In [32]:
top_weathered_granite_data

{'id': 'f16c98bf5bd2a6dc69224136fd1001ba',
 'speckle_type': 'Base',
 'totalChildrenCount': 0,
 '@Top Granite': {'id': '7d0cafafbe643e9f19d65110837407ab',
  'speckle_type': 'Base',
  'totalChildrenCount': 0,
  '@{0}': [{'id': '488d748c5197c1278e89928d3199c5f2',
    'speckle_type': 'Objects.Geometry.Point',
    'totalChildrenCount': 0,
    'applicationId': None,
    'units': 'm',
    'x': 837757.5,
    'y': 819782.5,
    'z': 37.75266771949828},
   {'id': 'f25f1f23382827f9856629e6f178fb37',
    'speckle_type': 'Objects.Geometry.Point',
    'totalChildrenCount': 0,
    'applicationId': None,
    'units': 'm',
    'x': 837762.5,
    'y': 819782.5,
    'z': 36.76180911436677},
   {'id': '0c743a55063e356346f48918298bf7ba',
    'speckle_type': 'Objects.Geometry.Point',
    'totalChildrenCount': 0,
    'applicationId': None,
    'units': 'm',
    'x': 837767.5,
    'y': 819782.5,
    'z': 35.770106283947825},
   {'id': '7ea27ff06a325d030cfceff11cd5cf30',
    'speckle_type': 'Objects.Geometry.P

In [33]:
len(obj["@Top Granite"]["@{0}"])
max_allowed = -7.5
num_corrected = 0
num_total = 0

for point in obj["@Top Granite"]["@{0}"]:
    if point["z"] > max_allowed:
        num_corrected +=   1
        point["z"] = max_allowed
    num_total += 1
    
num_corrected, num_total
        

(5893, 63874)

In [35]:
hash = operations.send(obj, [transport])


In [36]:
client.commit.create(
        branch_name="geo/ground-model/top-granite",
        stream_id=stream_id,
        object_id=hash,
        message=f"Corrected geol models at {dt.datetime.now()}",
    )

'9a7caa9444'